In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.12.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In [2]:
include(joinpath("../src/reduceTSC.jl"))
include(joinpath("../src/fileHandling.jl"))
include(joinpath("../src/isolate3Lines.jl"))
include(joinpath("../src/matroid_realization.jl"))
include(joinpath("../src/Jacobian_Criterion.jl"))

realization_space_2_singular_locus (generic function with 1 method)

In [3]:
#make [r] a basis, not useful here
function standardize_matroid(M)
    Bs = bases(M)
    
    ref = Bs[1]
    
    n = length(matroid_groundset(M))

    G = symmetric_group(n)
    
    t = vcat(ref,setdiff(1:n,ref))
    
    x = G(t)^(-1)
    
    new_Bs = [on_tuples(b,x) for b in Bs]
    
    new_M = matroid_from_bases(new_Bs,n)
    
    return new_M
    
end

#higgs lifs of M, not useful here
function Higgs(M)
    
    r = rank(M)
    gr = matroid_groundset(M)
    n = length(gr)
    
    Bs = [b for b in subsets(gr,r+1) if rank(M,b) == r]
    
    newM = matroid_from_bases(Bs,n)
    
    return newM
    
end



#obtain bases of flag matroid of the form [i-1]\cup{j}, (section 5.1 of Corey, Olarte)
function flag_bases_matrix_coordinates(Bs)
    coord_bases = [b for b in Bs if issubset(1:length(b)-1,b)]
    inds = [[length(b),maximum(b)] for b in coord_bases if !(length(b) == maximum(b))]
    
    return sort(inds)
    
end

#polynomial ring indexed by elements above.
function make_flag_polynomial_ring(Bs::Vector{Vector{Int}}, F::AbstractAlgebra.Ring)
    
    MC = flag_bases_matrix_coordinates(Bs)
    #R, x = PolynomialRing(F, :"x"=>MC)
    R, x = PolynomialRing(F, length(MC))
    xdict = Dict{Vector{Int}, MPolyElem}([MC[i] => x[i] for i in 1:length(MC)])
    return R, x, xdict
end

#obtain flag matroid from matrix. We assume that the columns indexed by sta
function flag_from_matrix(A)
    
    mats = []
    
    n = length(A[:,1])
    
    for i in 1:n
        
        Qi = matroid_from_matrix_columns(A[1:i,:])
        
        push!(mats,Qi)
        
    end
    
    return mats
end

function flag_bases(T)
    
    Bs = reduce(union,[bases(T[i]) for i in 1:length(T)])
    
    return Bs
    
end

function flag_nonbases(T)
    
    Bs = reduce(union,[nonbases(T[i]) for i in 1:length(T)])
    
    return Bs
    
end

function flag_var_matrix(T,F,Ring,dict)
    
    Bs = flag_bases(T)
    
   # Ring,x,dict = make_flag_polynomial_ring(Bs::Vector{Vector{Int}}, F::AbstractAlgebra.Ring)
    
    n = length(matroid_groundset(T[1]))
    
    r = maximum([rank(t) for t in T])
    
    X = MatrixSpace(Ring,r,n)
    
    XX = X(0)
    
    dict_keys = Vector([key for key in keys(dict)])
    
    for k in dict_keys
        XX[k[1],k[2]] = dict[[k[1],k[2]]]
    end
    
    for i in 1:r
        
        XX[i,i] = 1
        
    end
    
    return XX
    
end

#function data_gens(X,Bs)
    
  #  data = unique([det(X[1:length(b),b]) for b in Bs])
    
 #   return data
#end


function data_gens(X,Bs)
    
    data = []
    
    for b in Bs
        
        t = det(X[1:length(b),b])
        
        if !(-t in data)
                
                push!(data,t)
                    
         end
                
     end
            
        return unique(data)
            
end

#function to produce statrum data from flag matroid. We assume that the 
#sandard flag ([i] is a constituent) is a flag basis
    
function flag_stratum_data(T,F)
    
    Bs = flag_bases(T)
    nBs = flag_nonbases(T)
    
    R,x,dict = make_flag_polynomial_ring(Bs,F)
    
    X = flag_var_matrix(T,F,R,dict)
    
    Igens = data_gens(X,nBs)
    
    Sgens = data_gens(X,Bs)
    
    return (Igens, Sgens, R,x,dict,X)
end

flag_stratum_data (generic function with 1 method)

In [ ]:
#reproduce example 5.2 in  Corey, Olarte

In [4]:
m1 = matroid_from_bases([[1],[2],[4]],4)
m2 = matroid_from_bases([[1,2],[2,4]],4)
m3 = matroid_from_bases([[1,2,3],[1,2,4],[2,3,4]],4)
m4 = matroid_from_bases([[1,2,3,4]],4)

M1 = [m1,m2,m3,m4]

flag_stratum_data

flag_stratum_data (generic function with 1 method)

In [5]:
dat1 = flag_stratum_data(M1,QQ)
dat1[6]#matrix

[1   x1   0   x2]
[0    1   0    0]
[0    0   1   x3]
[0    0   0    1]

In [6]:
dat1[1]#ideal gens

1-element Vector{Any}:
 0

In [7]:
dat1[2]#semigroup gens

4-element Vector{Any}:
 1
 x1
 x2
 x3

In [72]:
#construct singular flag matroid stratum. 
#We start with a realization of the matroid in the paper. 
#This realization corresponds to a singular point in the realization space.

In [10]:
#matroid with singular TSC in paper
NB = [[1,2,6],[1,2,8],[1,3,5],[1,3,7],[1,5,7],
      [1,6,8],[1,9,12],[2,4,5],[2,4,9],[2,5,9],
      [2,6,8],[2,7,11],[3,4,6],[3,5,7],[3,8,9],
      [3,10,12],[4,5,9],[4,7,8],[4,10,11],[5,6,10],
      [8,11,12]]

Q = matroid_from_nonbases(NB, 12)

MR2 = reduce_ideal_full(new_matroid_realization_space(Q,[1,2,3,4];F =QQ,saturate = true))

X = MR2.representation_matrix
#setup 
Qs, s = QQ["s"];
K, (i, r3) = number_field([s^2 + 1, s^2 - 3], "a") # i = sqrt(-1) and r3 = sqrt(3)

R_og = MR2.ambient_ring
R, (x,y) = K["x", "y"]
phi = hom(R_og, R, a->a,  [x,y])
U = phi.(MR2.inequations);
X = phi.(MR2.representation_matrix)

q1 = [(3-i*r3)//3, (1-i*r3)//2]
q2 = [(3+i*r3)//3, (1+i*r3)//2]


A3 = matrix(R,[evaluate(X[i,j],q1) for i in 1:3, j in 1:12])#realization of matroid

I = identity_matrix(R,12)#

Acat = vcat(A3,I[4:12,:])#this matrix corrsponds to a full flag of subspaces in FL(12).

[1   0   0   1   1   1   -1//2*a1*a2 - 1//2                    1                    1      -1//3*a1*a2 + 1   -1//2*a1*a2 - 1//2      -1//3*a1*a2 + 1]
[0   1   0   1   0   1                    0   -1//2*a1*a2 + 1//2   -1//2*a1*a2 + 1//2   -1//2*a1*a2 + 1//2   -1//6*a1*a2 - 1//2   -1//2*a1*a2 + 1//2]
[0   0   1   1   1   0   -1//2*a1*a2 + 1//2                    0                    1    1//6*a1*a2 + 1//2   -1//2*a1*a2 + 1//2                    1]
[0   0   0   1   0   0                    0                    0                    0                    0                    0                    0]
[0   0   0   0   1   0                    0                    0                    0                    0                    0                    0]
[0   0   0   0   0   1                    0                    0                    0                    0                    0                    0]
[0   0   0   0   0   0                    1                    0                    0               

In [11]:
Tcat = flag_from_matrix(Acat)
FScat = flag_stratum_data(Tcat,K)

(Any[0, x1*x11 - x3*x10, x5*x13 - x6*x12, x7*x16 - x9*x14, x13*x23 - x16*x20, -x1*x18 + x2*x17, -x1*x20 + x6*x17, -x2*x20 + x6*x18, -x4*x22 + x8*x19, -x1*x13*x18 - x2*x10*x20 + x2*x13*x17 + x6*x10*x18, -x1*x12*x19 + x4*x12*x17 + x5*x10*x19, x1*x14*x22 - x1*x15*x21 - x7*x10*x22 + x7*x15*x17 + x8*x10*x21 - x8*x14*x17, x2*x11*x21 + x3*x14*x18 - x7*x11*x18, x5*x15*x23 - x5*x16*x22 - x8*x12*x23 + x9*x12*x22], Any[1, x1, x2, x3, x4, x5, x6, x7, x8, x9  …  -x1*x19 + x4*x17, x19, -x11*x18, x3*x18, -x1*x11*x18 + x2*x11*x17 + x3*x10*x18, -x11*x17, x3*x17, x10*x18, x18, x17], Multivariate polynomial ring in 23 variables over non-simple number field, AbstractAlgebra.Generic.MPoly{NfAbsNSElem}[x1, x2, x3, x4, x5, x6, x7, x8, x9, x10  …  x14, x15, x16, x17, x18, x19, x20, x21, x22, x23], Dict{Vector{Int64}, MPolyRingElem}([3, 10] => x21, [1, 8] => x5, [1, 6] => x3, [3, 5] => x18, [1, 11] => x8, [2, 12] => x16, [1, 9] => x6, [2, 8] => x12, [3, 12] => x23, [2, 6] => x11…), [1 0 0 x1 x2 x3 x4 x5 x6 x7 

In [13]:
redcat = reduce_ideal_full(FScat[1],FScat[2],FScat[3],FScat[4],false)

(AbstractAlgebra.Generic.MPoly{NfAbsNSElem}[x10^4*x22*x23^3 + x10^3*x15*x17*x23^3 - 2*x10^3*x16*x17*x22*x23^2 + 2*x10^2*x16^2*x17^2*x22*x23 - x10*x16^3*x17^3*x22 + x15*x16^3*x17^4], AbstractAlgebra.Generic.MPoly{NfAbsNSElem}[x16, x8, x17, x20, x11, x10, x18, x19, x12, x10*x22 - x15*x17  …  x10^4*x22^2*x23^2 - x10^3*x15*x17*x22*x23^2 + 2*x10^2*x15*x16*x17^2*x22*x23 - x10*x15*x16^2*x17^3*x22 + x15^2*x16^2*x17^4, x10*x23 - a1*x16*x17, x10*x23 + a1*x16*x17, x10^4*x22*x23^3 - 2*x10^3*x16*x17*x22*x23^2 + 2*x10^2*x16^2*x17^2*x22*x23 - x10*x16^3*x17^3*x22 + x15*x16^3*x17^4, x10^3*x22*x23^2 - x10^2*x16*x17*x22*x23 + 1//2*x10*x16^2*x17^2*x22 - 1//2*x15*x16^2*x17^3, x10^4*x22*x23^3 - x10^3*x16*x17*x22*x23^2 + x10^2*x16^2*x17^2*x22*x23 - x10*x16^3*x17^3*x22 + x15*x16^3*x17^4, x10^2*x22*x23^2 + x15*x16*x17^2*x23 - x16^2*x17^2*x22, x10*x22 + x15*x17, x10^3*x22*x23^2 - x10^2*x16*x17*x22*x23 - x15*x16^2*x17^3, x10*x23 + x16*x17], Multivariate polynomial ring in 23 variables over non-simple number fiel

In [14]:
Icat = ideal(redcat[3],redcat[1])#note this is a principle ideal.

ideal(x10^4*x22*x23^3 + x10^3*x15*x17*x23^3 - 2*x10^3*x16*x17*x22*x23^2 + 2*x10^2*x16^2*x17^2*x22*x23 - x10*x16^3*x17^3*x22 + x15*x16^3*x17^4)

In [16]:
satcat = stepwise_saturation(Icat,redcat[2])#check saturation. 

ideal(x10^4*x22*x23^3 + x10^3*x15*x17*x23^3 - 2*x10^3*x16*x17*x22*x23^2 + 2*x10^2*x16^2*x17^2*x22*x23 - x10*x16^3*x17^3*x22 + x15*x16^3*x17^4)

In [17]:
Icat == satcat

true

In [18]:
#generator of ideal factors 
Dict(factor(redcat[1][1]))

Dict{AbstractAlgebra.Generic.MPoly{NfAbsNSElem}, Int64} with 3 entries:
  x10^2*x22*x23 + x10*x15*x17*x23 - x10*x16*x17*x22 + x15*x16*x17^2 => 1
  x10*x23 + (1//2*a1*a2 - 1//2)*x16*x17                             => 1
  x10*x23 + (-1//2*a1*a2 - 1//2)*x16*x17                            => 1

In [20]:
JJ = ideal(redcat[3],redcat[1])+ideal(redcat[3],[derivative(redcat[1][1],i) for i in 1:length(redcat[4])])

ideal(x10^4*x22*x23^3 + x10^3*x15*x17*x23^3 - 2*x10^3*x16*x17*x22*x23^2 + 2*x10^2*x16^2*x17^2*x22*x23 - x10*x16^3*x17^3*x22 + x15*x16^3*x17^4, 4*x10^3*x22*x23^3 + 3*x10^2*x15*x17*x23^3 - 6*x10^2*x16*x17*x22*x23^2 + 4*x10*x16^2*x17^2*x22*x23 - x16^3*x17^3*x22, x10^3*x17*x23^3 + x16^3*x17^4, -2*x10^3*x17*x22*x23^2 + 4*x10^2*x16*x17^2*x22*x23 - 3*x10*x16^2*x17^3*x22 + 3*x15*x16^2*x17^4, x10^3*x15*x23^3 - 2*x10^3*x16*x22*x23^2 + 4*x10^2*x16^2*x17*x22*x23 - 3*x10*x16^3*x17^2*x22 + 4*x15*x16^3*x17^3, x10^4*x23^3 - 2*x10^3*x16*x17*x23^2 + 2*x10^2*x16^2*x17^2*x23 - x10*x16^3*x17^3, 3*x10^4*x22*x23^2 + 3*x10^3*x15*x17*x23^2 - 4*x10^3*x16*x17*x22*x23 + 2*x10^2*x16^2*x17^2*x22)

In [21]:
isone(JJ)

false

The potential issue with using the above is that the kth constituent has k-3 coloops. Maybe this this doesn't matter if the matroid polytope is full dim. If we can use this, I think its the best since it could let us reduce the case to studying the first 3 constituents.

In [24]:
#additional flag with slightly more interesting matroid

Acat2 = Acat

Acat2[4,6]=1
Acat2[5,8] = 1
Acat2[6,9] =1
Acat2[7,10] = 1
Acat2[8,11] =1
Acat2[9,12] = 1
Acat2[10,11] =1


[1   0   0   1   1   1   -1//2*a1*a2 - 1//2                    1                    1      -1//3*a1*a2 + 1   -1//2*a1*a2 - 1//2      -1//3*a1*a2 + 1]
[0   1   0   1   0   1                    0   -1//2*a1*a2 + 1//2   -1//2*a1*a2 + 1//2   -1//2*a1*a2 + 1//2   -1//6*a1*a2 - 1//2   -1//2*a1*a2 + 1//2]
[0   0   1   1   1   0   -1//2*a1*a2 + 1//2                    0                    1    1//6*a1*a2 + 1//2   -1//2*a1*a2 + 1//2                    1]
[0   0   0   1   0   1                    0                    0                    0                    0                    0                    0]
[0   0   0   0   1   0                    0                    1                    0                    0                    0                    0]
[0   0   0   0   0   1                    0                    0                    1                    0                    0                    0]
[0   0   0   0   0   0                    1                    0                    0               

In [25]:
for i in 1:12
    print(Acat2[i,:],"\n")
    end

[1 0 0 1 1 1 -1//2*a1*a2-1//2 1 1 -1//3*a1*a2+1 -1//2*a1*a2-1//2 -1//3*a1*a2+1]
[0 1 0 1 0 1 0 -1//2*a1*a2+1//2 -1//2*a1*a2+1//2 -1//2*a1*a2+1//2 -1//6*a1*a2-1//2 -1//2*a1*a2+1//2]
[0 0 1 1 1 0 -1//2*a1*a2+1//2 0 1 1//6*a1*a2+1//2 -1//2*a1*a2+1//2 1]
[0 0 0 1 0 1 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0 1 0 0]
[0 0 0 0 0 0 0 1 0 0 1 0]
[0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 1 1 0]
[0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 1]


In [27]:
T = flag_from_matrix(Acat2)
FS = flag_stratum_data(T,K)

(Any[0, x1*x11 - x3*x10, x5*x13 - x6*x12, x7*x16 - x9*x14, x13*x23 - x16*x20, -x1*x18 + x2*x17, -x1*x20 + x6*x17, -x2*x20 + x6*x18, -x4*x22 + x8*x19, -x1*x13*x18 - x2*x10*x20 + x2*x13*x17 + x6*x10*x18  …  -x2*x25*x26*x27 + x5*x26*x27, x2*x24*x25*x26*x27 - x5*x24*x26*x27, x10*x24*x26*x29 - x11*x26*x29 + x13*x29 - x16, -x10*x24*x26*x28*x29 + x11*x26*x28*x29 - x13*x28*x29 + x16*x28, x10*x24*x26*x27*x29 - x11*x26*x27*x29 + x13*x27*x29 - x16*x27, x10*x24*x26*x27*x28*x29 - x11*x26*x27*x28*x29 + x13*x27*x28*x29 - x16*x27*x28, -x10*x24*x25*x26*x28*x29 + x11*x25*x26*x28*x29 - x13*x25*x28*x29 + x16*x25*x28, -x10*x24*x25*x26*x27*x28*x29 + x11*x25*x26*x27*x28*x29 - x13*x25*x27*x28*x29 + x16*x25*x27*x28, -x10*x24*x26*x29*x30 + x11*x26*x29*x30 - x13*x29*x30 + x16*x30, -x10*x24*x26*x27*x29*x30 + x11*x26*x27*x29*x30 - x13*x27*x29*x30 + x16*x27*x30], Any[1, x1, x2, x3, x4, x5, x6, x7, x8, x9  …  x1*x24*x26*x29*x30 - x3*x26*x29*x30 + x6*x29*x30 - x9*x30, x17*x24*x26*x29*x30 + x20*x29*x30 - x23*x30, x24*

In [28]:
FS2 = flag_stratum_data(T,K)

(Any[0, x1*x11 - x3*x10, x5*x13 - x6*x12, x7*x16 - x9*x14, x13*x23 - x16*x20, -x1*x18 + x2*x17, -x1*x20 + x6*x17, -x2*x20 + x6*x18, -x4*x22 + x8*x19, -x1*x13*x18 - x2*x10*x20 + x2*x13*x17 + x6*x10*x18  …  -x2*x25*x26*x27 + x5*x26*x27, x2*x24*x25*x26*x27 - x5*x24*x26*x27, x10*x24*x26*x29 - x11*x26*x29 + x13*x29 - x16, -x10*x24*x26*x28*x29 + x11*x26*x28*x29 - x13*x28*x29 + x16*x28, x10*x24*x26*x27*x29 - x11*x26*x27*x29 + x13*x27*x29 - x16*x27, x10*x24*x26*x27*x28*x29 - x11*x26*x27*x28*x29 + x13*x27*x28*x29 - x16*x27*x28, -x10*x24*x25*x26*x28*x29 + x11*x25*x26*x28*x29 - x13*x25*x28*x29 + x16*x25*x28, -x10*x24*x25*x26*x27*x28*x29 + x11*x25*x26*x27*x28*x29 - x13*x25*x27*x28*x29 + x16*x25*x27*x28, -x10*x24*x26*x29*x30 + x11*x26*x29*x30 - x13*x29*x30 + x16*x30, -x10*x24*x26*x27*x29*x30 + x11*x26*x27*x29*x30 - x13*x27*x29*x30 + x16*x27*x30], Any[1, x1, x2, x3, x4, x5, x6, x7, x8, x9  …  x1*x24*x26*x29*x30 - x3*x26*x29*x30 + x6*x29*x30 - x9*x30, x17*x24*x26*x29*x30 + x20*x29*x30 - x23*x30, x24*

In [29]:
FS2[1]

74-element Vector{Any}:
 0
 x1*x11 - x3*x10
 x5*x13 - x6*x12
 x7*x16 - x9*x14
 x13*x23 - x16*x20
 -x1*x18 + x2*x17
 -x1*x20 + x6*x17
 -x2*x20 + x6*x18
 -x4*x22 + x8*x19
 -x1*x13*x18 - x2*x10*x20 + x2*x13*x17 + x6*x10*x18
 -x1*x12*x19 + x4*x12*x17 + x5*x10*x19
 x1*x14*x22 - x1*x15*x21 - x7*x10*x22 + x7*x15*x17 + x8*x10*x21 - x8*x14*x17
 x2*x11*x21 + x3*x14*x18 - x7*x11*x18
 ⋮
 x7*x16*x24*x25*x26 - x9*x14*x24*x25*x26
 x2*x25*x27 - x5*x27
 -x2*x25*x26*x27 + x5*x26*x27
 x2*x24*x25*x26*x27 - x5*x24*x26*x27
 x10*x24*x26*x29 - x11*x26*x29 + x13*x29 - x16
 -x10*x24*x26*x28*x29 + x11*x26*x28*x29 - x13*x28*x29 + x16*x28
 x10*x24*x26*x27*x29 - x11*x26*x27*x29 + x13*x27*x29 - x16*x27
 x10*x24*x26*x27*x28*x29 - x11*x26*x27*x28*x29 + x13*x27*x28*x29 - x16*x27*x28
 -x10*x24*x25*x26*x28*x29 + x11*x25*x26*x28*x29 - x13*x25*x28*x29 + x16*x25*x28
 -x10*x24*x25*x26*x27*x28*x29 + x11*x25*x26*x27*x28*x29 - x13*x25*x27*x28*x29 + x16*x25*x27*x28
 -x10*x24*x26*x29*x30 + x11*x26*x29*x30 - x13*x29*x30 + x16*x30


In [31]:
red = reduce_ideal_full(FS2[1],FS2[2],FS2[3],FS2[4],false)

(AbstractAlgebra.Generic.MPoly{NfAbsNSElem}[x11^4*x18^3*x22*x25^3 - 2*x11^3*x12*x17*x18^2*x22*x24*x25^2 + x11^3*x15*x17*x18^3*x24*x25^3 + 2*x11^2*x12^2*x17^2*x18*x22*x24^2*x25 - x11*x12^3*x17^3*x22*x24^3 + x12^3*x15*x17^4*x24^4], AbstractAlgebra.Generic.MPoly{NfAbsNSElem}[x11, x17, x8, x12, x23, x18, x19, x11*x22 - x15*x17*x24, x11^2*x18*x22*x25 - 1//2*x11*x12*x17*x22*x24 + 1//2*x12*x15*x17^2*x24^2, x21  …  x18*x25*x28 + x22, x29, x11^7*x18^4*x21*x22^3*x25^4*x26*x29 - x11^6*x12*x17*x18^4*x19*x22^2*x24*x25^4*x26*x27*x28*x29 - x11^6*x12*x17*x18^3*x21*x22^3*x24*x25^3*x26*x29 + x11^6*x12*x18^4*x21*x22^2*x23*x25^4*x28 + x11^6*x12*x18^3*x21*x22^3*x23*x25^3 + x11^6*x15*x17*x18^4*x19*x22^2*x24*x25^4*x26*x27*x29 + 2*x11^5*x12^2*x17^2*x18^3*x19*x22^2*x24^2*x25^3*x26*x27*x28*x29 - 2*x11^5*x12^2*x17^2*x18^3*x21*x22^2*x24^2*x25^3*x26*x28*x29 - x11^5*x12^2*x17*x18^3*x19*x22^2*x23*x24*x25^3*x27*x28 - x11^5*x12^2*x17*x18^3*x21*x22^2*x23*x24*x25^3*x28 - 2*x11^5*x12^2*x17*x18^2*x21*x22^3*x23*x24*x25^2 -

In [32]:
Dict(factor(red[1][1]))

Dict{AbstractAlgebra.Generic.MPoly{NfAbsNSElem}, Int64} with 3 entries:
  x11^2*x18*x22*x25 - x11*x12*x17*x22*x24 + x11*x15*x17*x18*x24*x25 + x12*… => 1
  x11*x18*x25 + (1//2*a1*a2 - 1//2)*x12*x17*x24                             => 1
  x11*x18*x25 + (-1//2*a1*a2 - 1//2)*x12*x17*x24                            => 1